> In the book Doubly Linked List
>  
> Here seperate lists of vertexes, faces, half-edges
>  
> "Pointers" simply indexes in appropriate list //best of my ideas yet


### Classes for vertexes, faces, half-edges

In [2]:
class Vertex:
    def __init__(self, coord, incident_edge):
        # tuple with coordinates
        self.coord = coord 
        
        # half-edge with this vertex as origin  -- not exactly sure how this should work
        self.incident_edge = incident_edge

        
        
class Face:
    def __init__(self, outer_component):
        # index of and half-edge on outer boundary
        self.outer = outer_component
        
        # list of indexes of one (outer) half-edge in every hole inside this face
        self.inner = []
        
    # adding hole (one of outer half_edges from hole inside this face)
    def add_hole(self, inner_component_half_edge):
        self.inner.append(inner_component_half_edge)
        
    #for the unbounded face, self.outer = None    -- should be default??
        
        
        
class Half_edge:
    def __init__(self, origin, twin, nxt, prv):
        # vertex this half_edge starts from
        self.origin = origin
        
        # opposite half_edge (from destination(this) to origin(this))
        self.twin = twin
        
        # face laying to the left of half_edge
        self.incident_face = None
        
        #next half_edge that is also a boundary to self.incident_face
        self.next = nxt
        
        #previous half_edge that is also a boundary to self.incident_face
        self.prev = prv
    
    def set_face(self, f):
        self.incident_face = f
        
    # self.destination = self.twin.origin hence needs not to be used
        

### Subdivision



In [3]:
# "Doubly Linked" Edge List
class Dles:
    def __init__(self):
        # list of tuples (vertex, id of element of subdivision)
        self.v_list = []
        # face list
        self.f_list = []
        # half_edge list
        self.he_list = []
        
    # self-recongnizing adding vertexes/faces/half_edges
    def add(self, x):
        if type(x).__name__ == 'Vertex':
            self.v_list.append(x)
        elif type(x).__name__ == 'Face':
            self.f_list.append(x)
        elif type(x).__name__ == 'Half_edge':
            self.he_list.append(x)
        

## Converting set of lists of edges into edge list


> This part converts given **lists of edges** only into a set of half_edges list and vertex list, creating the face list later
> 
> Changing to list of points (vertexes) izi if neccessary
>  
> A subdivision is a polygon with possible holes (polygons) inside
>  
> Each vertice has exactly two adjacent edges


In [38]:
# subdivision is list of lists of edges
# subdivision = [polygon1, polygon2, ...] - one is outer boundary, rest are boundaries of holes inside it

def convert( subdivision ): 

    edge_list = Dles()
    
    # index for half_edges
    edge_index = 0
    e0 = 0
    # neccessary to keep indexes in modulo(length*2) + e0
    def m(x):
         return (x-e0)%(len(polygon)*2) +e0
    # index for vertexes
    vertex_index = 0
    v0 = 0
    def v_i(x):
        return (x-v0)%(len(polygon))+v0
        
    for polygon in subdivision:
        
        for edge in polygon:
            
            # adding vertex v - beginning of given edge - end is not added
            edge_list.add(Vertex(edge[0], m(edge_index)))
                          
            # adding incident half_edge (starting in v) at index he_i
            edge_list.add(Half_edge(v_i(vertex_index), m(edge_index+1) , m(edge_index+2), m(edge_index-2)))
                          
            #adding the twin of added half_edge with at index he_i+1
            edge_list.add(Half_edge(v_i(vertex_index+1), m(edge_index), m(edge_index-1), m(edge_index+3)))
            
            vertex_index+=1
            edge_index+=2    # i - edge, i+1 - twin edge
            
        e0 = edge_index
        v0 = vertex_index
        
    return edge_list

In [39]:
#test

polygon1 = [[(1,1), (1,2)], [(1,2),(2,2)], [(2,2), (2,1)], [(2,1), (1,1)]]
# polygon2 = [[(0,0), (0,5)], [(0,5), (5,5)], [(5,5),(5,0)], [(5,0), (3,1)], [(3,1),(0,0)]]
# polygon1 = [[(1,1), (1,2)], [(1,2),(2,2)], [(2,2), (1,1)]]
polygon2 = [[(0,0), (0,5)], [(0,5), (5,5)], [(5,5),(0,0)]]
subdivision = [polygon1, polygon2]
# subdivision = [polygon1]

list_of_edges = convert(subdivision)

def print_dles(L):
    for v in L.v_list:
        print(v.coord, '   inc. edge:', v.incident_edge)
    print()
    i = 0
    for e in L.he_list:
        print(i, 'twin:', e.twin, " prev,next: ", e.prev, ',', e.next, 'or: ', e.origin)
        i+=1
        
print_dles(list_of_edges)

4
e0 8
3
e0 14
(1, 1)    inc. edge: 0
(1, 2)    inc. edge: 2
(2, 2)    inc. edge: 4
(2, 1)    inc. edge: 6
(0, 0)    inc. edge: 8
(0, 5)    inc. edge: 10
(5, 5)    inc. edge: 12

0 twin: 1  prev,next:  6 , 2 or:  0
1 twin: 0  prev,next:  3 , 7 or:  1
2 twin: 3  prev,next:  0 , 4 or:  1
3 twin: 2  prev,next:  5 , 1 or:  2
4 twin: 5  prev,next:  2 , 6 or:  2
5 twin: 4  prev,next:  7 , 3 or:  3
6 twin: 7  prev,next:  4 , 0 or:  3
7 twin: 6  prev,next:  1 , 5 or:  0
8 twin: 9  prev,next:  12 , 10 or:  4
9 twin: 8  prev,next:  11 , 13 or:  5
10 twin: 11  prev,next:  8 , 12 or:  5
11 twin: 10  prev,next:  13 , 9 or:  6
12 twin: 13  prev,next:  10 , 8 or:  6
13 twin: 12  prev,next:  9 , 11 or:  4


#### !!
> Cause I'll forget
> 
> To find later one boundary cycle in he_list, remember origin of the first edge you look at and follow nexts until you get to that origin